# Definição do Problema

Prever o turnover (rotatividade) de empregados com base em várias características, como demografia, profissão, características psicológicas e outros atributos relacionados ao trabalho.

In [ ]:
import pandas as pd
import numpy as np

turnover_data = pd.read_csv('turnover.csv')

# Visão Geral dos Dados

Ao visualizar os dados vemos que existem 16 atributos, dos quais "EVENT" sera o atributo target da nossa predição.

**stag**: Tempo t quando ocorreu o turnover ou tempo de censura do estudo.
**event**: Indica se o evento ocorreu no tempo t.  
**gender**: Gênero do funcionário.  
**age**: Idade do funcionário.  
**industry**: Indústria de atuação do funcionário.  
**profession**: Profissão do funcionário.  
**traffic**: De qual canal o funcionário ingressou para a empresa  
**coach**: Presença de um coach  
**head_gender**: Gênero do supervisor ao qual o funcionario responde  
**greywage**: se o empregador paga apenas uma pequena quantia de salário acima do salário mínimo (white).  
**way**: Forma que o funcionário se desloca para o escritório.  
**extraversion**: Escala segundo o teste Big5.  
**independ**: Escala segundo o teste Big5.  
**selfcontrol**: Escala segundo o teste Big5.  
**anxiety**: Escala segundo o teste Big5.  
**novator**: Escala segundo o teste Big5.  

In [ ]:
turnover_data

# Pré-processamento de Dados

## Tratamento de Nulos

In [ ]:
pd.DataFrame({
    'Null Count': turnover_data.isnull().sum().fillna(0).astype(int),
    'Dtype': turnover_data.dtypes
})

Em uma primeira análise vemos que nao existem valores nulos na nossa base, assim como os tipos de valores existentes são:

quantitativo continuo = 7  
qualitativo nominal = 8

Tambem vemos que na base nao existem valores nulos ou faltando

## Valores repetidos

In [ ]:
duplicated_rows = turnover_data[turnover_data.duplicated()]
print(f"Numero de linhas duplicadas = {duplicated_rows.shape[0]}")

In [ ]:
turnover_data = turnover_data.drop_duplicates()

## Outliers

Para determinar os outliers é necessário fazer um estudo sobre os quartis e ver como estes se comportam, porque assim é possivel saber se existe uma tendencia a possuir mais valores de um tipo do que outro e assim analisar os outliers

In [ ]:
turnover_data.describe()

Devido à natureza de que outliers so irão existir em categorias numéricas, diversas colunas podem ser ignoradas. Além disso, com exceção de stag, todas as colunas apresentam uma distribuição igualitária onde o valor do quartil se encaixa de maneira consideravelmente próxima à porcentagem do valor máximo em relação ao mínimo. Portanto a coluna stag é a unica a qual deve ser analisado a existência de outliers.

Nesta situação uma boa técnica é o uso do z-score, o qual mede o desvio padrão de uma instância em relação à media, assim podemos captar outliers.

In [ ]:
# Cálculo dos z-scores para todas as colunas numéricas do DataFrame
z_scores = (turnover_data['stag'] - turnover_data['stag'].mean()) / turnover_data['stag'].std()

# Recuperando as linhas que possuem outliers baseado no z-score 3
outliers = z_scores[(z_scores > 3) | (z_scores < -3)].index

len(outliers)

In [ ]:
turnover_data = turnover_data.drop(outliers)
turnover_data.describe()

O outlier 2.5 se mostrou adequado pois foi o que mais aproximou o valor discrepante do padrão de quartis

## Balanceamento

Para saber se uma base de dados é balanceada deve-se olhar a destribuição entre o atributo target, que neste caso é a coluna **event**

In [ ]:
import matplotlib.pyplot as plt

# Análise da distribuição da variável alvo
event_distribution = turnover_data['event'].value_counts()

# Plotando a distribuição
plt.figure(figsize=(8, 6))
event_distribution.plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribuição da Variável Alvo (event)')
plt.xlabel('Classe')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.show()

event_distribution

Dado que a base nao é desbalanceada, basta seguir em frente

# Transformação de Dados

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})

# Seleciona apenas colunas não numéricas
non_numeric_columns = turnover_data.select_dtypes(exclude=['float64', 'int64'])

plt.figure(figsize=(6, 5))

# Para cada coluna categórica
for col in non_numeric_columns.columns:
    values = turnover_data[col].value_counts().tolist()
    plt.plot(range(len(values)), values, marker='.', label=col)

plt.xlabel('Índice dos Atributos')
plt.ylabel('Quantidade')
plt.title('Distribuição de atributos categoricos')
plt.legend(title='Colunas Categóricas', loc='upper right')
plt.grid(True, which='both', linestyle=':', linewidth=1)
plt.show()

Analisando a quantidade de possíveis instâncias por coluna chega-se a conclusão de que existem:  
3 colunas binarizáveis (gender, head_gender e greywage)  
2 colunas pequenas o suficiente para realizar encode ou one hot (coach e way)  
3 colunas que precisarão de um estudo de caso para decidir como tratá-las  

## Industry

In [ ]:
# Análise da variável 'industry'
industry_distribution = turnover_data['industry'].value_counts(normalize=True)

# Plotando a distribuição
plt.figure(figsize=(6, 6))
industry_distribution.plot(kind='barh', color='cornflowerblue')
plt.title('Distribuição da Variável "industry"')
plt.xlabel('Proporção')
plt.ylabel('Indústria')
plt.show()

Industry possui diversos valores com uma destribuição considerável em cada feature.  
Por isso, a estratégia adotada será realizar um encoding onde o valor de industry será substituído pela porcentagem de quantos desses resultados terminaram em turnover, assim ainda permitindo que o modelo retire insights valiosos da feature.

In [ ]:
# Calculando a relação de target (mean encoding) para a variável 'industry'
industry_target_relation = turnover_data.groupby('industry')['event'].mean()

# Ordenando os valores para melhor visualização
industry_target_relation_sorted = industry_target_relation.sort_values(ascending=False)

# Plotando a relação de target da variável 'industry'
plt.figure(figsize=(6, 6))
industry_target_relation_sorted.plot(kind='barh', color='lightseagreen')
plt.title('Relação de Target da Variável "industry"')
plt.xlabel('Média de Turnover (Event)')
plt.ylabel('Indústria')
plt.show()

Dado uma relação satisfatória que separa os atributos, o tornam válido para a utilização da técnica, portanto pode-se substituir na tabela original o novo valor

In [ ]:
# Aplicando a codificação target (mean encoding) para a variável 'industry' no dataset original
turnover_data['industry_encoded'] = turnover_data['industry'].map(industry_target_relation)

# Substituindo a coluna 'industry' pela coluna 'industry_encoded'
turnover_data.drop('industry', axis=1, inplace=True)
turnover_data.rename(columns={'industry_encoded': 'industry'}, inplace=True)

# Exibindo as primeiras linhas do dataframe após a substituição
turnover_data.head()

## Profession

In [ ]:
# Análise da variável 'profession'
profession_distribution = turnover_data['profession'].value_counts(normalize=True)

# Plotando a distribuição
plt.figure(figsize=(6, 6))
profession_distribution.plot(kind='barh', color='lightgreen')
plt.title('Distribuição da Variável "profession"')
plt.xlabel('Proporção')
plt.ylabel('Profissão')
plt.show()

Diferente de Industry, Profession possui uma grande quantidade de instâncias aglomeradas em uma única classe de maneira desbalanceada, portanto os valores de classes como PR não irão apresentar um resultado relevante para o nosso modelo. Para tratar isso as classes com uma quantidade de instâncias menor que "Etc" são colocadas dentro desta mesma classe

In [ ]:
# Calculando a distribuição das categorias em 'profession'
profession_distribution_original = turnover_data['profession'].value_counts(normalize=True)

# Identificando as categorias que serão agrupadas em "etc"
threshold_original = profession_distribution_original['etc']
categories_to_group_original = profession_distribution_original[profession_distribution_original <= threshold_original].index

# Agrupando categorias menos frequentes em "etc"
turnover_data['profession_grouped'] = turnover_data['profession'].apply(lambda x: 'etc' if x in categories_to_group_original else x)

# Verificando a distribuição após o agrupamento
updated_profession_distribution = turnover_data['profession_grouped'].value_counts(normalize=True)

# Plotando a distribuição da variável 'profession_grouped'
plt.figure(figsize=(6, 6))
updated_profession_distribution.plot(kind='bar', color='lightcoral')
plt.title('Distribuição da Variável "profession_grouped"')
plt.xlabel('Profissão Agrupada')
plt.ylabel('Proporção')
plt.xticks(rotation=0)
plt.show()

Estando as instâncias categorizadas em uma pequena quantidade de classes, é possível utilizar One Hot Encoding para classificar o atributo

In [ ]:
# Realizando one-hot encoding para 'profession_grouped'
profession_onehot = pd.get_dummies(turnover_data['profession_grouped'], prefix='profession', drop_first=False)

# Concatenando o one-hot encoding ao dataframe original
turnover_data = pd.concat([turnover_data, profession_onehot], axis=1)

# Removendo as colunas 'profession' e 'profession_grouped' já que não são mais necessárias
turnover_data.drop(['profession', 'profession_grouped'], axis=1, inplace=True)

# Exibindo as primeiras linhas do dataframe atualizado
turnover_data.head()

## Traffic

In [ ]:
# Análise da variável 'traffic'
traffic_distribution = turnover_data['traffic'].value_counts(normalize=True)

# Plotando a distribuição
plt.figure(figsize=(6, 6))
traffic_distribution.plot(kind='barh', color='orchid')
plt.title('Distribuição da Variável "traffic"')
plt.xlabel('Proporção')
plt.ylabel('Fonte de Recrutamento')
plt.show()

Traffic acaba tendo o mesmo problema de Industry por não ter um desbalanceamento demasiadamente grande, por isso é sugerido seguir pelo mesmo caminho de realizar um mean encoding

In [ ]:
# Calculando mean encoding para a variável 'traffic'
traffic_mean_encoding = turnover_data.groupby('traffic')['event'].mean()

# Aplicando a codificação para visualização
turnover_data['traffic_encoded'] = turnover_data['traffic'].map(traffic_mean_encoding)

# Exibindo as primeiras linhas da coluna original 'traffic' e da coluna codificada 'traffic_encoded'
traffic_comparison = turnover_data[['traffic', 'traffic_encoded']].drop_duplicates().sort_values(by='traffic_encoded', ascending=False)

# Substituindo a coluna 'traffic' pela coluna 'traffic_encoded'
turnover_data.drop('traffic', axis=1, inplace=True)
turnover_data.rename(columns={'traffic_encoded': 'traffic'}, inplace=True)

# Exibindo as primeiras linhas do dataframe após a substituição
turnover_data.head()

## Coach

Coach pode ser considerado um atributo binário a partir de certo ponto de vista, mas para a pesquisa foi diferenciado entre o coach ser um agente externo ou o próprio treinador do funcionàrio, para isso é seguro afirmar que podemos realizar um encoding contando que "my head" seja colocado ao lado de "yes" como um supertipo, mas não importando a ordem entre os dois

In [ ]:
# Calculando a distribuição dos valores na coluna 'coach'
coach_distribution = turnover_data['coach'].value_counts(normalize=True)

# Plotando a distribuição dos valores
plt.figure(figsize=(6, 6))
coach_distribution.plot(kind='bar', color='lightgreen')
plt.title('Coach distribution')
plt.xlabel('Coach')
plt.ylabel('Proporção')
plt.xticks(rotation=0)
plt.show()

coach_distribution

In [ ]:
# Criando a coluna 'coach_encoded'
turnover_data['coach_encoded'] = turnover_data['coach'].map({'yes': 2, 'no': 0, 'my head': 1})

# Removendo a coluna original 'coach'
turnover_data.drop('coach', axis=1, inplace=True)

## Way

Os valores de way não são possíveis de se relacionar diretamente por não possuirem uma cardinalidade, portanto deve-se usar uma estratégia de separá-los em atributos binarizados, já que possuem apenas 3 tipos de instâncias

In [ ]:
# Calculando a distribuição dos valores na coluna 'way'
coach_distribution = turnover_data['way'].value_counts(normalize=True)

# Plotando a distribuição dos valores
plt.figure(figsize=(6, 6))
coach_distribution.plot(kind='bar', color='lightgreen')
plt.title('Way distribution')
plt.xlabel('Way')
plt.ylabel('Proporção')
plt.xticks(rotation=0)
plt.show()

coach_distribution

In [ ]:
# Realizando one-hot encoding para a variável 'way'
way_onehot = pd.get_dummies(turnover_data['way'], prefix='way', drop_first=False)

# Concatenando o one-hot encoding ao dataframe original
turnover_data = pd.concat([turnover_data, way_onehot], axis=1)

# Removendo a coluna original 'way' já que não é mais necessária
turnover_data.drop('way', axis=1, inplace=True)

# Exibindo as primeiras linhas do dataframe após o one-hot encoding
turnover_data.head()

## Binarização de atributos com 2 categorias

In [ ]:
# Identificando colunas com exatamente 2 valores únicos
binary_columns = [col for col in turnover_data.columns if turnover_data[col].nunique() == 2]

# Binarizando as colunas identificadas
for col in binary_columns:
    unique_vals = turnover_data[col].unique()
    turnover_data[col] = turnover_data[col].replace({unique_vals[0]: 0, unique_vals[1]: 1})

# Exibindo as primeiras linhas do dataframe após a binarização
turnover_data.head()

## Normalização \ Padronização

In [ ]:
# Exibindo as primeiras linhas do dataframe após a normalização
turnover_data.describe().loc[['min', 'max']]

Devido aos valores estarem contidos dentro de um range relativamente curto, não se torna obrigatório realizar a normalização, mas ainda assim se torna benéfico para a qualidade do nosso modelo para evitar peso desnecessário em stag

In [ ]:
# Identificando colunas que precisam ser normalizadas
columns_to_normalize = [col for col in turnover_data.columns 
                        if turnover_data[col].min() < 0 or turnover_data[col].max() > 1]

# Normalizando as colunas identificadas
for col in columns_to_normalize:
    turnover_data[col] = (turnover_data[col] - turnover_data[col].min()) / \
                                 (turnover_data[col].max() - turnover_data[col].min())

turnover_data.describe().loc[['min', 'max']]

# Análise Exploratória de Dados

Atraves da analise de correlação entre os elementos podemos ver se o crescimento das variaveis segue direta ou inversamente o crescimento de determinada variavel alvo, seguindo uma escala que começa de 1 se for diretamente proporcional ou até -1 se for inversamente proporcional.
Nem sempre estas relações significam que no treinamento do modelo elas não possuirão relevancia, mas ainda sim é um grande indicativo e muitos insights podem ser retirados delas.

In [ ]:
import seaborn as sns

# Calculando a matriz de correlação
correlation_matrix = turnover_data.corr()

# Plotando a matriz de correlação usando um heatmap
plt.figure(figsize=(20, 15))
ax = sns.heatmap(correlation_matrix, cmap='RdBu_r', fmt='.2f', linewidths=0.5)

# Adicionar anotações numéricas manualmente
for i in range(correlation_matrix.shape[0]):
    for j in range(correlation_matrix.shape[1]):
        value = correlation_matrix.iloc[i, j]
        # Ajustando a cor baseada no valor da correlação
        text_color = 'black' if -0.5 < value < 0.5 else 'white'
        ax.text(j + 0.5, i + 0.5, f'{value:.2f}', ha='center', va='center', color=text_color, fontsize=14)

plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

E possível identificar que existe correlação consideravelmente alta entre os valores de way_car para way_bus assim como de professional_etc para professional_HR

mas para o caso de way_car, a relação entre este valor para way_foot fornece um insight importante para perceber que por mesmo que exista uma correlação próxima, ele ainda é um forte candidato a realizar desempates em relação a terceiras e quartas variáveis.

Ambos os dois casos naturalmente possuem uma correlação devido ao fato de serem mutuamente excludentes durante seu processo de encoding, mas ainda assim seus valores interferem na avaliação final do modelo

#### Agora correlacionando exclusivamente os valores em relação ao evento em questão temos os seguintes:

In [ ]:
turnover_data.corr()['event'].sort_values(ascending=False).drop('event')

Levando isso em conta e testes que foram realizados no modelo final, concluimos que gender, profession_IT, novator e coach_encoded diminuem o score geral dos modelos. Isso juntamente com sua pontuação proxima de 0 na analise de correlação nos faz decidir tira-los da tabela.

In [ ]:
# Removendo as variáveis relacionadas a 'profession', 'way' e 'greywage'
columns_to_drop = ['gender'] + ['profession_IT'] + ['novator'] + ['coach_encoded']
turnover_data = turnover_data.drop(columns=columns_to_drop)

# Verificando as primeiras linhas do dataframe após a remoção das variáveis
turnover_data.head()

# Divisão dos Dados

Como a validação do modelo sera feito através da utilização de cross validation então apenas separaremos o dados entre X e y
A separação exata em folds sera realizada automaticamente durante a utilização do grid-search

In [ ]:
X = turnover_data.drop('event', axis=1)
y = turnover_data['event']

# Criação dos Modelos

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Criando os modelos
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
nb = GaussianNB()
gb = GradientBoostingClassifier()
lr = LogisticRegression()
ab = AdaBoostClassifier()
kn = KNeighborsClassifier()
sv = SVC()
nn = MLPClassifier()

## Definição de hiperparâmetros para o uso do Grid Search

In [ ]:
# Decision Tree
param_dist_dt = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'criterion': ['gini', 'entropy']
}

# Random Forest
param_dist_rf = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Naive Bayes
param_dist_nb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

# Gradient Boosting
param_dist_gb = {
    'loss': ['log_loss', 'exponential'],
    'learning_rate': [0.001, 0.1, 1],
    'n_estimators': [50, 100, 200, 500],
    'criterion': ['friedman_mse', 'squared_error']
}

# Regressão Logistica
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
param_dist_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'max_iter': [50, 100, 500, 1000, 5000],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    
}

# AdaBoost
param_dist_ab = {
    'n_estimators': [50, 100, 200, 500],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'algorithm': ['SAMME', 'SAMME.R']
}

# KNeighbors
param_dist_kn = {
    'n_neighbors': [3, 5, 8],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 30, 50],
    'p': [1, 2]
}

param_dist_sv = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['poly', 'sigmoid'],
    'gamma': ['scale', 'auto'],
    'coef0': [0.0, 0.5, 1.0],
    'class_weight': [None, 'balanced'],
}

param_dist_nn = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

## Treinamento dos modelos utilizando o Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo a instancia de Grid Search
grid_search_dt = GridSearchCV(dt, param_grid=param_dist_dt, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_rf = GridSearchCV(rf, param_grid=param_dist_rf, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_gb = GridSearchCV(gb, param_grid=param_dist_gb, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_ab = GridSearchCV(ab, param_grid=param_dist_ab, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_kn = GridSearchCV(kn, param_grid=param_dist_kn, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_nb = GridSearchCV(nb, param_grid=param_dist_nb, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_lr = GridSearchCV(lr, param_grid=param_dist_lr, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_sv = GridSearchCV(sv, param_grid=param_dist_sv, cv=10, n_jobs=1, verbose = 1, error_score='raise')
grid_search_nn = GridSearchCV(nn, param_grid=param_dist_nn, cv=10, n_jobs=1, verbose = 1, error_score='raise')

# Ajustando os valores
print('------------ Decision Tree ------------')
grid_search_dt.fit(X, y)
print('------------ Random Forest ------------')
grid_search_rf.fit(X, y)
print('---------- Gradient Boosting ----------')
grid_search_gb.fit(X, y)
print('-------------- Ada Boost --------------')
grid_search_ab.fit(X, y)
print('------------- K Neighbors -------------')
grid_search_kn.fit(np.ascontiguousarray(X), np.ascontiguousarray(y))
print('------------- Naive Bayes -------------')
grid_search_nb.fit(X, y)
print('--------- Regressão Logistica ---------')
grid_search_lr.fit(X, y)
print('-------- Support Vector Machine -------')
grid_search_sv.fit(X, y)
print('---------- Neural Network MLP ---------')
grid_search_nn.fit(X, y)

### Melhores valores encontrados no Grid Search

In [ ]:
print(f'Decision Tree          = {grid_search_dt.best_params_}')
print(f'Random Forest          = {grid_search_rf.best_params_}')
print(f'Gradient Boosting      = {grid_search_gb.best_params_}')
print(f'Ada Boost              = {grid_search_ab.best_params_}')
print(f'K-Neighbors            = {grid_search_kn.best_params_}')
print(f'Naive Bayes            = {grid_search_nb.best_params_}')
print(f'Regressão Logistica    = {grid_search_lr.best_params_}')
print(f'Support Vector Machine = {grid_search_sv.best_params_}')
print(f'Neural Network MLP     = {grid_search_nn.best_params_}')

## Cross validação com os algoritmos devidamente treinados

In [ ]:
from sklearn.model_selection import cross_val_predict

# Retirando as previsões encontradas no cross-validation do melhor grid search
dt_y_pred_cv = cross_val_predict(grid_search_dt.best_estimator_, X, y, cv=10)
rf_y_pred_cv = cross_val_predict(grid_search_rf.best_estimator_, X, y, cv=10)
gb_y_pred_cv = cross_val_predict(grid_search_gb.best_estimator_, X, y, cv=10)
ab_y_pred_cv = cross_val_predict(grid_search_ab.best_estimator_, X, y, cv=10)
kn_y_pred_cv = cross_val_predict(grid_search_kn.best_estimator_, np.ascontiguousarray(X), np.ascontiguousarray(y), cv=10)
nb_y_pred_cv = cross_val_predict(grid_search_nb.best_estimator_, X, y, cv=10)
lr_y_pred_cv = cross_val_predict(grid_search_lr.best_estimator_, X, y, cv=10)
sv_y_pred_cv = cross_val_predict(grid_search_sv.best_estimator_, X, y, cv=10)
nn_y_pred_cv = cross_val_predict(grid_search_nn.best_estimator_, X, y, cv=10)

### Retirada do score encontrado nos modelos com cross validação

In [ ]:
from sklearn.model_selection import cross_val_score

modelos = [
    ('DT', grid_search_dt.best_estimator_),
    ('RF', grid_search_rf.best_estimator_),
    ('GB', grid_search_gb.best_estimator_),
    ('AB', grid_search_ab.best_estimator_),
    ('KNN', grid_search_kn.best_estimator_),
    ('NB', grid_search_nb.best_estimator_),
    ('LR', grid_search_lr.best_estimator_),
    ('SVM', grid_search_sv.best_estimator_),
    ('MLP', grid_search_nn.best_estimator_),
]

acuracias_modelos = {i: cross_val_score(modelo, np.ascontiguousarray(X), y, cv=5, scoring='accuracy') for i, modelo in modelos}
acuracias_modelos

# Matriz de confusão

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from matplotlib.patches import Rectangle

conf_matrix_dt = confusion_matrix(y, dt_y_pred_cv)
conf_matrix_rf = confusion_matrix(y, rf_y_pred_cv)
conf_matrix_gb = confusion_matrix(y, gb_y_pred_cv)
conf_matrix_ab = confusion_matrix(y, ab_y_pred_cv)
conf_matrix_kn = confusion_matrix(y, kn_y_pred_cv)
conf_matrix_nb = confusion_matrix(y, nb_y_pred_cv)
conf_matrix_lr = confusion_matrix(y, lr_y_pred_cv)
conf_matrix_sv = confusion_matrix(y, sv_y_pred_cv)
conf_matrix_nn = confusion_matrix(y, nn_y_pred_cv)

matrices = [conf_matrix_dt, conf_matrix_rf, conf_matrix_gb, conf_matrix_ab, conf_matrix_kn, conf_matrix_nb, conf_matrix_lr, conf_matrix_sv, conf_matrix_nn]
titles = ['Decision Tree', 'Random Forest', 'Gradient Boosting', 'Ada Boost', 'K-Nearest', 'Naive Bayes', 'Logistic Regression', 'SVM', 'Neural Network']

# 3 linhas, 3 colunas
fig, axarr = plt.subplots(3, 3, figsize=(6, 7.5))  

# Para facilitar a iteração, vamos achatá-lo (flatten)
axes = axarr.flatten()

classes = ['No', 'Yes']  # Classes

# Agora iteramos sobre as matrizes e os títulos como antes, mas usando o axes achatado
for i, (matrix, title) in enumerate(zip(matrices, titles)):
    axes[i].imshow(matrix, cmap="Blues", aspect='auto')
    axes[i].set_title(title, fontsize=14)
    
    # Adicionar anotações numéricas com tamanho de fonte maior
    for j in range(matrix.shape[0]):
        for k in range(matrix.shape[1]):
            axes[i].text(k, j, str(matrix[j, k]), ha='center', va='center', fontsize=16)
    
    axes[i].axis('off')
    rect = Rectangle((0,0), 1, 1, edgecolor='black', facecolor='none', transform=axes[i].transAxes, linewidth=3)
    axes[i].add_patch(rect)

    # Rótulos das linhas (classe verdadeira)
    if i == 0 or i == 3 or i == 6:
        for j, cls in enumerate(classes):
            axes[i].text(-0.6, j, cls, ha='right', va='center', color='black', transform=axes[i].transData)
        # Rótulos das colunas (classe prevista)
    for j, cls in enumerate(classes):
        axes[i].text(j, 1.9, cls, ha='center', va='bottom', color='black', transform=axes[i].transData)

plt.tight_layout()
plt.show()

A partir de uma visão geral da matriz de confusão conseguimos perceber que existe uma proporção entre a predição correta de valores sim/nao, mostrando que a precisão irá se aproximar bastante do recall

# Validação do modelo

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred, model_name):
    return {
        'Model': model_name,
        'Accuracy': round(accuracy_score(y_true, y_pred), 3),
        'Precision': round(precision_score(y_true, y_pred), 3),
        'Recall': round(recall_score(y_true, y_pred), 3),
        'F1-Score': round(f1_score(y_true, y_pred), 3)
    }

# Computando a validacao do modelo
dt_report = calculate_metrics(y, dt_y_pred_cv, 'Decision Tree')
rf_report = calculate_metrics(y, rf_y_pred_cv, 'Random Forest')
gb_report = calculate_metrics(y, gb_y_pred_cv, 'Naive Bayes')
ab_report = calculate_metrics(y, ab_y_pred_cv, 'Ada Boost')
kn_report = calculate_metrics(y, kn_y_pred_cv, 'K-nearest')
nb_report = calculate_metrics(y, nb_y_pred_cv, 'Gradient Boosting')
lr_report = calculate_metrics(y, lr_y_pred_cv, 'Logistic Regression')
sv_report = calculate_metrics(y, sv_y_pred_cv, 'SVM')
nn_report = calculate_metrics(y, nn_y_pred_cv, 'Neural Network')

reports = [dt_report, rf_report, gb_report, ab_report, kn_report, nb_report, lr_report, sv_report, nn_report]

summary_df = pd.DataFrame(reports)
summary_df

# Teste estatistico T

In [ ]:
from scipy import stats

# Crie um DataFrame vazio com as colunas de modelos
model_names = list(acuracias_modelos.keys())
comparison_df = pd.DataFrame(index=model_names, columns=model_names)

# Preencha a diagonal principal com "N/A" ou algum outro valor apropriado
comparison_df.values[[i for i in range(len(model_names))], [i for i in range(len(model_names))]] = "N/A"

# Realize as comparações entre modelos e preencha a tabela com os valores P
for i in range(len(model_names)):
    for j in range(i + 1, len(model_names)):
        t_statistic, p_value = stats.ttest_ind(acuracias_modelos[model_names[i]], acuracias_modelos[model_names[j]])
        comparison_df.at[model_names[i], model_names[j]] = round(p_value, 3)
        comparison_df.at[model_names[j], model_names[i]] = round(p_value, 3)

# Visualize a tabela de comparação
print(comparison_df)

Dentre os modelos propostos o metodo de **Gradient Boosting** foi aquele que teve o maior destaque juntamente do **Random Forest**. Para decidir entre qual é mais importante, seja Precisão ou Recall é importante se analisar o contexto no qual a empresa que está realizando a análise esta. 

Priorizar a **Precisão** esta correlacionado a ideia de diminuir o número de "alarmes falsos", o que é especialmente útil caso a empresa queira realizar ações caras ou significativas para intervir, como o oferecimento de bônus ou promoções para reter talentos.

Priorizar o **Recall** se torna importante caso a empresa queira evitar perder funcionários devido a rotatividade por exemplo. Ter um recall baixo significa que muitos funcionários que realmente saíram não foram identificados pelo modelos, isso se torna crítico em setores ou funções onde a perda de talentos é muito custosa para a organização (por exemplo em casos altamente especializados).

# Interpretabilidade dos modelos gerados

Modelos baseados em árvore podem ser usados para se analisar quais atributos tiveram mais importância no treinamento, a partir de uma análise de quais foram os nós raiz gerados.

In [ ]:
# Para Random Forest, por exemplo:
importances_dt = grid_search_dt.best_estimator_.feature_importances_
importances_rf = grid_search_rf.best_estimator_.feature_importances_
importances_gb = grid_search_gb.best_estimator_.feature_importances_

# Criando o DataFrame
df_importances = pd.DataFrame({
    'feature': X.columns,
    'importance_dt': importances_dt,
    'importance_rf': importances_rf,
    'importance_gb': importances_gb
})

# Crie uma coluna com o valor acumulado das importâncias
df_importances['total_importance'] = df_importances['importance_dt'] + df_importances['importance_rf'] + df_importances['importance_gb']

# Ordene o DataFrame com base na importância total, em ordem decrescente
df_importances = df_importances.sort_values(by='total_importance', ascending=False)

# Valores acumulados
df_importances['importance_rf_acc'] = df_importances['importance_dt'] + df_importances['importance_rf']
df_importances['importance_gb_acc'] = df_importances['importance_rf_acc'] + df_importances['importance_gb']

# Configurar figura e eixos
plt.figure(figsize=(6, 6))

colors = sns.color_palette("Pastel1", n_colors=3)
# Plotando importância para cada modelo
plt.barh(df_importances['feature'], df_importances['importance_gb_acc'], color=colors[2], label='Gradient Boosting')
plt.barh(df_importances['feature'], df_importances['importance_rf_acc'], color=colors[1], label='Random Forest')
plt.barh(df_importances['feature'], df_importances['importance_dt'], color=colors[0], label='Decision Tree')
plt.gca().invert_yaxis()


plt.legend(loc='lower right')
plt.show()

Com estes dados podemos chegar a algumas conclusões importantes, sendo a principal o tempo de permanência do funcionário o principal fator de influência para saber se ele vai ficar na empresa ou não. Outras inferências podem ser feitas como a importância do tipo de industria sobre o tipo de cargo, o que indica que o clima e cultura empresarial são fortes candidatos para a análise de sobrevivência de funcionários. Por outro lado características pessoais de auto controle e ansiedade se mostraram também importantes, sendo estas provavelmente relacionadas a resiliência do indivíduo. É válido ressaltar que o meio de contratação pode acabar também indicando a existência de fatores culturais vindos daquela plataforma. E por fim que a idade é um fator crucial para saber a resiliência de um funcionário em uma vaga.